# Quantifying Similarity for Textual Studies

The following notebook demonstrates how to implement the formulas discussed in Robert Turnbull, 'The Textual Character of Codex Sinaiticus Arabicus and its Family', forthcoming.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

If a manuscript has $A$ unambiguous agreements with a comparison text and $D$ disagreements, the basic estimate of similarity is $\hat p = \frac{A}{A+D}$.


In [0]:
def similarity(agreements, disagreements):
    return agreements/(agreements+disagreements)

# Credible Interval

The similarity distribution can be modeled using Bayesian Inference by a Beta distribution:
$$p \sim Beta(A+1,D+1)$$
Where $A$ is the number of agreements and $D$ is the number of disagreements.
NB. This assumes a uniform prior, i.e. $B(0,0)$.


A credible interval (analogous to 'confidence interval' in frequentist statistics) with lower and upper bounds ($\overleftarrow { p }$ and $\overrightarrow { p }$ respectively) can be calculated through the quantiles of this distribution:

$$\int _{ 0 }^{  \overleftarrow { p }  }{ Beta(A+1,D+1)dp=\frac { \alpha  }{ 2 }  } $$
$$\int _{ \overrightarrow { p } }^{  1  }{ Beta(A+1,D+1)dp=\frac { \alpha  }{ 2 }  } $$


The Python code below is derived from:
Cameron, Ewan. ‘[On the Estimation of Confidence Intervals for Binomial Population Proportions in Astronomy: The Simplicity and Superiority of the Bayesian Approach](https://www.cambridge.org/core/journals/publications-of-the-astronomical-society-of-australia/article/on-the-estimation-of-confidence-intervals-for-binomial-population-proportions-in-astronomy-the-simplicity-and-superiority-of-the-bayesian-approach/AB407B3E5A201AB2CF351C44F96700B3).’ *Publications of the Astronomical Society of Australia* 28.2 (2013): 128–39.


In [0]:
import scipy.stats.distributions as dist

def similarity_upper_bound(agreements, disagreements, alpha=0.05):
    return dist.beta.ppf(
            1.0 - 0.5 * alpha, 
            agreements + 1, 
            disagreements + 1 )

def similarity_lower_bound(agreements, disagreements, alpha=0.05):
    return dist.beta.ppf(
            0.5 * alpha, 
            agreements + 1, 
            disagreements + 1 )

## Examples

In [0]:
agreements = 42
disagreements = 67

print( "Similarity = %.1f%%" % (similarity(agreements, disagreements ) * 100) )
print( "Lower bound = %.1f%%" % (similarity_lower_bound(agreements, disagreements ) * 100) )
print( "Upper bound = %.1f%%" % (similarity_upper_bound(agreements, disagreements ) * 100) )


Similarity = 38.5%
Lower bound = 29.9%
Upper bound = 47.9%


Create a sample data frame in Pandas to use as an example.

In [0]:
import pandas as pd
import random

random.seed(0)

data = []
max_value = 200
manuscript_count = 15

for manuscript in range(manuscript_count):
    agreements = random.randint(1,max_value+1)
    disagreemnets = random.randint(1,max_value+1)

    data += [[manuscript, agreements, disagreemnets]]
    
# Create the dataframe
df = pd.DataFrame(data, columns = ['Manuscript', 'Agreements', 'Disagreements']) 
  
# Display dataframe
df 


,Manuscript,Agreements,Disagreements
0,0,99,195
1,1,108,11
2,2,67,131
3,3,125,104
4,4,201,78
5,5,123,92
6,6,150,56
7,7,130,36
8,8,73,36
9,9,194,25


Now the calculate the similarity with lower and upper bounds for each row.

In [0]:

df['Similarity'] = df.apply(lambda row:similarity(row['Agreements'], row['Disagreements']), axis=1)
df['Similarity_Lower_Bound'] = df.apply(lambda row:similarity_lower_bound(row['Agreements'], row['Disagreements']), axis=1)
df['Similarity_Upper_Bound'] = df.apply(lambda row:similarity_upper_bound(row['Agreements'], row['Disagreements']), axis=1)

# Display dataframe. 
df


,Manuscript,Agreements,Disagreements,Similarity,Similarity_Lower_Bound,Similarity_Upper_Bound
0,0,99,195,0.336735,0.285131,0.392620
1,1,108,11,0.907563,0.841901,0.947252
2,2,67,131,0.338384,0.276110,0.406921
3,3,125,104,0.545852,0.481069,0.609095
4,4,201,78,0.720430,0.664971,0.769800
5,5,123,92,0.572093,0.505179,0.636430
6,6,150,56,0.728155,0.663522,0.784286
7,7,130,36,0.783133,0.714268,0.838950
8,8,73,36,0.669725,0.576721,0.750923
9,9,194,25,0.885845,0.836833,0.921329


# Ranking

The rank estimate ($\hat r$) is:

$$\hat r=1+\sum _{ m \in G }{ \left[ {\hat p }_{ b }>{ \hat p }_{ m }  \right]  }$$

Where $m$ is an element of a set manuscripts (which excludes manuscript being ranked). The square Iverson brackets represents the count for which the inequality holds.

In [0]:
df['Rank'] = df['Similarity'].rank(ascending=False)
df.sort_values(by=['Rank'], ascending=True)

,Manuscript,Agreements,Disagreements,Similarity,Similarity_Lower_Bound,Similarity_Upper_Bound,Rank
14,14,187,19,0.907767,0.860375,0.939979,1.0
1,1,108,11,0.907563,0.841901,0.947252,2.0
9,9,194,25,0.885845,0.836833,0.921329,3.0
12,12,155,38,0.803109,0.741205,0.852969,4.0
7,7,130,36,0.783133,0.714268,0.838950,5.0
13,13,80,26,0.754717,0.664575,0.826659,6.0
6,6,150,56,0.728155,0.663522,0.784286,7.0
4,4,201,78,0.720430,0.664971,0.769800,8.0
10,10,159,65,0.709821,0.647146,0.765297,9.0
8,8,73,36,0.669725,0.576721,0.750923,10.0


To calculate the high and low bounds for the rank ($\overleftarrow r$ and $\overrightarrow r$) for manuscript $a$:

$$\overleftarrow { r } =1+\sum _{ m \in G }{ \left[ Pr\left( { p }_{ b }>{ p }_{ m } \right) <\frac { \alpha  }{ 2 }  \right]  } $$
$$\overrightarrow { r } =1+\sum _{ m \in G }{ \left[ Pr\left( { p }_{ b }>{ p }_{ m } \right) < 1- \frac { \alpha  }{ 2 }  \right]  } $$


Where $g$ is an element of the set of $N$ manuscripts (which excludes manuscript $a$). The probability $Pr\left( { p }_{ b }>{ p }_{ m } \right)$ is calculated using the formula demonstrated by Evan Miller in his post ([Formulas for Bayesian A/B Testing](https://www.evanmiller.org/bayesian-ab-testing.html#binary_ab)):

$$Pr(p_B > p_A) = \sum_{i=0}^{\alpha_B-1}{\frac{B(\alpha_A+i, \beta_B + \beta_A)}{(\beta_B+i)B(1+i,\beta_B)B(\alpha_A,\beta_A)}}$$

where $B$ is the Beta function, $p_A \sim Beta(\alpha_A, \beta_A)$ and $p_B \sim Beta(\alpha_B, \beta_B)$. In our case, $p$ for any manuscript $x$ is modeled as $p_x \sim Beta(agreements_x+1, disagreements_x+1)$ and so $\alpha_x = agreements_x + 1$ and $\beta_x = disagreements_x + 1$.


The function below is based on Miller's implementation in Julia.
NB. In this case, the product above is calculated using the logarithms in case of overflow issues. 

In [0]:
import math
from  scipy.special import betaln

def probability_B_beats_A(alpha_distA, beta_distA, alpha_distB, beta_distB):
    total = 0.0
    for i in range(int(alpha_distB)):
        # Calculate the sum 
        total += math.exp(betaln(alpha_distA+i, beta_distB+beta_distA) - math.log(beta_distB+i) - betaln(1+i, beta_distB) - betaln(alpha_distA, beta_distA))
    return total

Now the 95% credible interval for the rank can be calculated.

In [0]:
def credible_rank(rowB, threshold):    
    count = 0
    
    for index, rowA in df.iterrows():
        # Skip if it is the same manuscript
        if rowA['Manuscript'] == rowB['Manuscript']:
            continue
        probability_this_row_beats_other = probability_B_beats_A( rowA['Agreements']+1, rowA['Disagreements']+1, rowB['Agreements']+1, rowB['Disagreements']+1  )
        if probability_this_row_beats_other < threshold:
            count += 1
    return count + 1

alpha = 0.05
df['Highest_rank'] = df.apply(lambda row:credible_rank(row, alpha*0.5), axis=1)
df['Lower_rank'] = df.apply(lambda row:credible_rank(row, 1.0-alpha*0.5), axis=1)
df.sort_values(by=['Rank'], ascending=True)

,Manuscript,Agreements,Disagreements,Similarity,Similarity_Lower_Bound,Similarity_Upper_Bound,Rank,Highest_rank,Lower_rank
14,14,187,19,0.907767,0.860375,0.939979,1.0,1,3
1,1,108,11,0.907563,0.841901,0.947252,2.0,1,3
9,9,194,25,0.885845,0.836833,0.921329,3.0,1,3
12,12,155,38,0.803109,0.741205,0.852969,4.0,4,7
7,7,130,36,0.783133,0.714268,0.838950,5.0,4,9
13,13,80,26,0.754717,0.664575,0.826659,6.0,4,10
6,6,150,56,0.728155,0.663522,0.784286,7.0,4,10
4,4,201,78,0.720430,0.664971,0.769800,8.0,5,10
10,10,159,65,0.709821,0.647146,0.765297,9.0,5,10
8,8,73,36,0.669725,0.576721,0.750923,10.0,6,11
